## diff models via weights

In [ ]:
import itertools

import torch
from transformers import AutoModelForCausalLM

torch.set_grad_enabled(False)


model_groups = [
    [
        "inclusionAI/AReaL-boba-2-8B-Open",
        "nvidia/AceReason-Nemotron-7B",
        "qihoo360/Light-R1-7B-DS",
        "Skywork/Skywork-OR1-7B",
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    ],
    [
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
        "nvidia/AceReason-Nemotron-1.1-7B",
        "open-r1/OpenR1-Distill-7B",
        "sail/Qwen2.5-Math-7B-Oat-Zero",
        "Qwen/Qwen2.5-Math-7B",
        "open-r1/Qwen2.5-Math-7B-RoPE-300k",
    ],
    [
        "agentica-org/DeepCoder-1.5B-Preview",
        "zwhe99/DeepMath-1.5B",
        "agentica-org/DeepScaleR-1.5B-Preview",
        "Nickyang/FastCuRL-1.5B-V3",
        "oumi-ai/MiniMath-R1-1.5B",
        "nvidia/Nemotron-Research-Reasoning-Qwen-1.5B",
        "GD-ML/Open-RS1",
        "knoveleng/Open-RS3",
        "RUC-AIBOX/STILL-3-1.5B-preview",
        "Zyphra/ZR1-1.5B",
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    ],
]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


def diff_weights(model1, model2) -> pd.DataFrame:
    records = []

    count_skipped = 0
    for (name1, param1), (name2, param2) in zip(model1.model.named_parameters(), model2.model.named_parameters()):
        if name1 != name2 or param1.size() != param2.size():
            count_skipped += 1
            continue
        diff = torch.linalg.norm(param1.data - param2.data)
        norm1 = torch.linalg.norm(param1.data)
        norm2 = torch.linalg.norm(param2.data)
        normalized_diff = diff / (0.5 * norm1 + 0.5 * norm2)
        normalized_diff1 = diff / norm1
        normalized_diff2 = diff / norm2
        ft_change = (param1.data - param2.data).abs().mean()  # https://arxiv.org/pdf/2403.20284
        records.append(
            {
                "name1": name1,
                "name2": name2,
                "diff": diff.item(),
                "normalized_diff": normalized_diff.item(),
                "normalized_diff1": normalized_diff1.item(),
                "normalized_diff2": normalized_diff2.item(),
                "ft_change": ft_change.item(),
            }
        )

    df = pd.DataFrame.from_records(records)
    if not df.empty and not (df["name1"] == df["name2"]).all():
        print("Layer names are not the same, might not match")
    print(f"Skipped {count_skipped} comparisons due to mismatching names or sizes")
    return df


def _layername_to_type(s: str) -> str:
    if s.startswith("layers"):
        return ".".join(s.split(".")[2:])
    else:
        return ".".join(s.split(".")[0:])


def _layername_to_layer(s: str) -> int:
    if s.startswith("layers"):
        return int(s.split(".")[1])
    else:
        return -1


def plot_diffs(df: pd.DataFrame, title: str = "", y: str = "diff", **kwargs):
    if not (df.name1 == df.name2).all():
        raise ValueError("Layers do not match.")

    df = df.copy()
    df["layer_type"] = df["name1"].apply(_layername_to_type)
    df["layer_idx"] = df["name1"].apply(_layername_to_layer)

    g = sns.relplot(data=df, x="layer_idx", y=y, col="layer_type", col_wrap=5, height=2, aspect=1.6, **kwargs)
    g.figure.suptitle(title, y=1.02)


In [ ]:
old_df = pd.read_csv("weight_diff.csv")
dfs = []

for i, model_group in enumerate(model_groups):
    MODELS = {}
    for model_name1, model_name2 in itertools.combinations(model_group, 2):
        if len(old_df.loc[(old_df.model1 == model_name1) & (old_df.model2 == model_name2) & (old_df.group == i)]) > 0:
            print(model_name1, model_name2, "already exists")
            continue

        try:
            model1 = MODELS.get(model_name1, None)
            if not model1:
                model1 = AutoModelForCausalLM.from_pretrained(
                    model_name1, torch_dtype=torch.bfloat16, device_map="cpu"
                )
                MODELS[model_name1] = model1

            model2 = MODELS.get(model_name2, None)
            if not model2:
                model2 = AutoModelForCausalLM.from_pretrained(
                    model_name2, torch_dtype=torch.bfloat16, device_map="cpu"
                )
                MODELS[model_name2] = model2

            df = diff_weights(model1, model2)
            plot_diffs(df, f"{model_name1} vs {model_name2}")

            df["model1"] = model_name1
            df["model2"] = model_name2
            df["group"] = i
            dfs.append(df)
        except Exception as e:
            print(model_name1, model_name2, e)


In [ ]:
combined = pd.concat(dfs, axis=0)
combined = pd.concat([old_df, combined], axis=0)

combined
combined.to_csv("weight_diff.csv")
